In [5]:
from pathlib import Path

RAW   = Path("../data/raw")
CLEAN = Path("../data/clean")
OUT   = Path("../outputs")
for p in (RAW, CLEAN, OUT): p.mkdir(parents=True, exist_ok=True)

assert (RAW/"311_noise.csv").exists(),  "Missing 311_noise.csv in data/raw"
assert (RAW/"sla_active.csv").exists(), "Missing sla_active.csv in data/raw"
print("Found raw files.")


Found raw files.


In [4]:
import re, numpy as np

TARGET_PREFIXES = ("103", "112")  # 103xx = Staten Island, 112xx = Brooklyn

def fix_zip(val):
    s = str(val)
    m = re.search(r"(\d{5})", s)
    return m.group(1) if m else np.nan

def is_target_zip(z):
    z = str(z)
    return (len(z) == 5) and (z[:3] in TARGET_PREFIXES)

def borough_from_zip(z):
    z = str(z)
    if z.startswith("103"): return "Staten Island"
    if z.startswith("112"): return "Brooklyn"
    return "Other"


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests


In [8]:
# Load the raw 311 file
noise_raw = pd.read_csv(RAW/"311_noise.csv", low_memory=False)
noise = noise_raw.copy()
noise.columns = [c.strip().lower().replace(" ", "_") for c in noise.columns]

# Auto-detect common column names (adjust if yours differ)
zip_col_candidates   = [c for c in noise.columns if c in ("incident_zip","zip","incident_address_zip","zipcode")]
type_col_candidates  = [c for c in noise.columns if c in ("complaint_type","complaint")]
date_col_candidates  = [c for c in noise.columns if "created" in c and "date" in c]  # e.g., created_date

if not zip_col_candidates:
    print("311 headers I see (first 30):", list(noise.columns)[:30])
    raise ValueError("Couldn't find a ZIP column in 311 CSV. Add the correct name to zip_col_candidates.")

zip_col  = zip_col_candidates[0]
type_col = type_col_candidates[0] if type_col_candidates else None
date_col = date_col_candidates[0] if date_col_candidates else None

# Keep only Noise complaints (safety)
if type_col:
    noise = noise[noise[type_col].str.startswith("Noise", na=False)].copy()

# Clean ZIP + keep only BK/SI ZCTAs
noise["zip"] = noise[zip_col].apply(fix_zip)
noise = noise[noise["zip"].apply(is_target_zip)].copy()

# Hour-of-day (for heatmap)
if date_col:
    noise["created_dt"] = pd.to_datetime(noise[date_col], errors="coerce")
    noise["hour"] = noise["created_dt"].dt.hour

print("Rows after filtering to BK + SI Noise:", len(noise))
noise.head()


Rows after filtering to BK + SI Noise: 217982


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,zip,created_dt,hour
1,66465967,10/14/2025 01:51:14 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11212.0,181 ROCKAWAY PARKWAY,...,NaN,NaN,NaN,NaN,40.662317,-73.921442,"(40.66231674694474, -73.92144177201318)",11212,2025-10-14 01:51:14,1
4,66471764,10/14/2025 01:50:02 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11214.0,1869 83 STREET,...,NaN,NaN,NaN,NaN,40.608101,-73.999384,"(40.60810108364675, -73.99938414563323)",11214,2025-10-14 01:50:02,1
11,66467429,10/14/2025 01:46:20 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11226.0,101 WOODRUFF AVENUE,...,NaN,NaN,NaN,NaN,40.653708,-73.962710,"(40.65370793721925, -73.96270993293973)",11226,2025-10-14 01:46:20,1
13,66464509,10/14/2025 01:45:52 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11212.0,270 MOTHER GASTON BOULEVARD,...,NaN,NaN,NaN,NaN,40.672770,-73.907673,"(40.672769616675254, -73.9076728513286)",11212,2025-10-14 01:45:52,1
29,66470371,10/14/2025 01:37:34 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,10309.0,168 MACGREGOR STREET,...,NaN,NaN,NaN,NaN,40.523635,-74.199475,"(40.52363478309076, -74.19947548745674)",10309,2025-10-14 01:37:34,1


In [10]:
# === Cell 4: Load & clean SLA (BK + SI) ===
sla_raw = pd.read_csv(RAW/"sla_active.csv", low_memory=False)
sla = sla_raw.copy()
sla.columns = [c.strip().lower().replace(" ", "_") for c in sla.columns]

# 1) Find the ZIP column (include 'zip_code')
sla_zip_candidates = [c for c in sla.columns if c in (
    "premise_zip", "premises_zip", "zip", "zipcode", "zip_code", "premises_zip_code"
)]
if not sla_zip_candidates:
    print("SLA headers I see (first 30):", list(sla.columns)[:30])
    raise ValueError("Couldn't find a ZIP column in SLA CSV. Add the correct name to sla_zip_candidates.")

zip_col_sla = sla_zip_candidates[0]

# 2) Clean ZIPs (ensure 5-digit strings even if the CSV stored integers)
sla["zip"] = sla[zip_col_sla].apply(fix_zip)

# 3) (Extra safety) Limit to Kings/Richmond counties if the column exists
if "premises_county" in sla.columns:
    sla = sla[sla["premises_county"].str.upper().isin(["KINGS", "RICHMOND"])]

# 4) Keep only Staten Island (103xx) and Brooklyn (112xx)
sla = sla[sla["zip"].apply(is_target_zip)].copy()

print("ZIP column used:", zip_col_sla)
print("Active licenses in BK + SI:", len(sla))
sla.head()


ZIP column used: zip_code
Active licenses in BK + SI: 134


,license_permit_id,premises_county,type,class,description,legalname,dba,actual_address_of_premises,additional_address_information,city,...,zip_code,original_issue_date,last_issue_date,effective_date,expiration_date,parent_license_id,legacy_serial_number,aka_address,georeference,zip
0,0340-22-106991,Richmond,1,340,Restaurant,MCMADD INC,GRIFF'S PLACE,702 NEW DORP LANE,NEW DROP LANE & HETT AVENUE,STATEN ISLAND,...,10306,04/01/2025,04/01/2025,09/01/2024,08/31/2026,NaN,1226590,NaN,POINT (-74.10047 40.56586),10306
1,0240-23-142229,Richmond,1,240,Restaurant,AVANTI PIZZA 1 INC,AVANTI PIZZA,170 NEW DORP LANE,NaN,STATEN ISLAND,...,10306,04/30/2025,04/30/2025,05/01/2025,04/30/2027,NaN,1334685,NaN,POINT (-74.11514 40.57335),10306
2,0340-21-116837,Richmond,1,340,Restaurant,SAKAI RAMEN INC,SAKAI RAMEN & IZAKAYA,"965 RICHMOND AVE, STORE F",NaN,STATEN ISLAND,...,10314,02/04/2025,02/04/2025,02/01/2024,01/31/2026,NaN,1320779,NaN,POINT (-74.14787 40.62349),10314
3,0240-23-139773,Richmond,1,240,Restaurant,SANDWICH AND PICKLE LLC,NaN,1949 RICHMOND AVE,STORE #3,STATEN ISLAND,...,10314,07/30/2025,07/30/2025,08/01/2025,07/31/2027,NaN,1353750,NaN,POINT (-74.16259 40.6004),10314
4,0240-23-142528,Richmond,1,240,Restaurant,AVOCADO SUSHI I INC,NaN,4906 ARTHUR KILL RD,STE B,STATEN ISLAND,...,10309,06/18/2025,06/18/2025,06/18/2025,01/31/2027,NaN,1347142,NaN,POINT (-74.23879 40.52253),10309


In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import re
from pathlib import Path

# Recreate helper stuff in case kernel restarted
RAW   = Path("../data/raw")
CLEAN = Path("../data/clean")
OUT   = Path("../outputs")
TARGET_PREFIXES = ("103", "112")  # 103xx Staten Island, 112xx Brooklyn

def fix_zip(val):
    s = str(val)
    m = re.search(r"(\d{5})", s)
    return m.group(1) if m else np.nan

def is_target_zip(z):
    z = str(z)
    return (len(z) == 5) and (z[:3] in TARGET_PREFIXES)

def borough_from_zip(z):
    z = str(z)
    if z.startswith("103"):
        return "Staten Island"
    if z.startswith("112"):
        return "Brooklyn"
    return "Other"


In [2]:
noise_raw = pd.read_csv(RAW/"311_noise.csv", low_memory=False)
noise = noise_raw.copy()
noise.columns = [c.strip().lower().replace(" ", "_") for c in noise.columns]

zip_col_candidates   = [c for c in noise.columns if c in ("incident_zip","zip","incident_address_zip","zipcode")]
type_col_candidates  = [c for c in noise.columns if c in ("complaint_type","complaint")]
date_col_candidates  = [c for c in noise.columns if ("created" in c and "date" in c)]

zip_col  = zip_col_candidates[0]
type_col = type_col_candidates[0] if type_col_candidates else None
date_col = date_col_candidates[0] if date_col_candidates else None

if type_col:
    noise = noise[noise[type_col].str.startswith("Noise", na=False)].copy()

noise["zip"] = noise[zip_col].apply(fix_zip)
noise = noise[noise["zip"].apply(is_target_zip)].copy()

if date_col:
    noise["created_dt"] = pd.to_datetime(noise[date_col], errors="coerce")
    noise["hour"] = noise["created_dt"].dt.hour

print("noise rows:", len(noise))
noise.head()

noise rows: 217982


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,zip,created_dt,hour
1,66465967,10/14/2025 01:51:14 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11212.0,181 ROCKAWAY PARKWAY,...,NaN,NaN,NaN,NaN,40.662317,-73.921442,"(40.66231674694474, -73.92144177201318)",11212,2025-10-14 01:51:14,1
4,66471764,10/14/2025 01:50:02 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11214.0,1869 83 STREET,...,NaN,NaN,NaN,NaN,40.608101,-73.999384,"(40.60810108364675, -73.99938414563323)",11214,2025-10-14 01:50:02,1
11,66467429,10/14/2025 01:46:20 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11226.0,101 WOODRUFF AVENUE,...,NaN,NaN,NaN,NaN,40.653708,-73.962710,"(40.65370793721925, -73.96270993293973)",11226,2025-10-14 01:46:20,1
13,66464509,10/14/2025 01:45:52 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11212.0,270 MOTHER GASTON BOULEVARD,...,NaN,NaN,NaN,NaN,40.672770,-73.907673,"(40.672769616675254, -73.9076728513286)",11212,2025-10-14 01:45:52,1
29,66470371,10/14/2025 01:37:34 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,10309.0,168 MACGREGOR STREET,...,NaN,NaN,NaN,NaN,40.523635,-74.199475,"(40.52363478309076, -74.19947548745674)",10309,2025-10-14 01:37:34,1


In [3]:
sla_raw = pd.read_csv(RAW/"sla_active.csv", low_memory=False)
sla = sla_raw.copy()
sla.columns = [c.strip().lower().replace(" ", "_") for c in sla.columns]

sla_zip_candidates = [c for c in sla.columns if c in (
    "premise_zip","premises_zip","zip","zipcode","zip_code","premises_zip_code"
)]
zip_col_sla = sla_zip_candidates[0]

sla["zip"] = sla[zip_col_sla].apply(fix_zip)

if "premises_county" in sla.columns:
    sla = sla[sla["premises_county"].str.upper().isin(["KINGS","RICHMOND"])]

sla = sla[sla["zip"].apply(is_target_zip)].copy()

print("sla rows:", len(sla))
sla.head()


sla rows: 134


,license_permit_id,premises_county,type,class,description,legalname,dba,actual_address_of_premises,additional_address_information,city,...,zip_code,original_issue_date,last_issue_date,effective_date,expiration_date,parent_license_id,legacy_serial_number,aka_address,georeference,zip
0,0340-22-106991,Richmond,1,340,Restaurant,MCMADD INC,GRIFF'S PLACE,702 NEW DORP LANE,NEW DROP LANE & HETT AVENUE,STATEN ISLAND,...,10306,04/01/2025,04/01/2025,09/01/2024,08/31/2026,NaN,1226590,NaN,POINT (-74.10047 40.56586),10306
1,0240-23-142229,Richmond,1,240,Restaurant,AVANTI PIZZA 1 INC,AVANTI PIZZA,170 NEW DORP LANE,NaN,STATEN ISLAND,...,10306,04/30/2025,04/30/2025,05/01/2025,04/30/2027,NaN,1334685,NaN,POINT (-74.11514 40.57335),10306
2,0340-21-116837,Richmond,1,340,Restaurant,SAKAI RAMEN INC,SAKAI RAMEN & IZAKAYA,"965 RICHMOND AVE, STORE F",NaN,STATEN ISLAND,...,10314,02/04/2025,02/04/2025,02/01/2024,01/31/2026,NaN,1320779,NaN,POINT (-74.14787 40.62349),10314
3,0240-23-139773,Richmond,1,240,Restaurant,SANDWICH AND PICKLE LLC,NaN,1949 RICHMOND AVE,STORE #3,STATEN ISLAND,...,10314,07/30/2025,07/30/2025,08/01/2025,07/31/2027,NaN,1353750,NaN,POINT (-74.16259 40.6004),10314
4,0240-23-142528,Richmond,1,240,Restaurant,AVOCADO SUSHI I INC,NaN,4906 ARTHUR KILL RD,STE B,STATEN ISLAND,...,10309,06/18/2025,06/18/2025,06/18/2025,01/31/2027,NaN,1347142,NaN,POINT (-74.23879 40.52253),10309


In [4]:
try:
    url = "https://api.census.gov/data/2023/acs/acs5/profile"
    params = {
        "get": "NAME,DP05_0001E",
        "for": "zip code tabulation area:*",
        "in": "state:36"  # NY
    }
    r = requests.get(url, params=params, timeout=5)  # short timeout
    r.raise_for_status()

    zcta = pd.DataFrame(r.json()[1:], columns=r.json()[0]).rename(
        columns={"DP05_0001E": "pop_total",
                 "zip code tabulation area": "zip"}
    )
    zcta["zip"] = zcta["zip"].astype(str).str.zfill(5)
    zcta["pop_total"] = pd.to_numeric(zcta["pop_total"], errors="coerce")

    zcta_target = zcta[zcta["zip"].apply(is_target_zip)].copy()
    print("✅ Population downloaded from Census.")
    print(zcta_target.head())

except Exception as e:
    print("⚠ Could not reach Census API. Using fallback ZIP list without population.")
    print("Error:", e)

    # fallback: just make something so we can keep going
    zips_we_saw = sorted(set(noise["zip"]).union(set(sla["zip"])))
    zcta_target = pd.DataFrame({
        "zip": zips_we_saw,
        "pop_total": np.nan  # we'll fill this later
    })

    print(zcta_target)


⚠ Could not reach Census API. Using fallback ZIP list without population.
Error: 400 Client Error:  for url: https://api.census.gov/data/2023/acs/acs5/profile?get=NAME%2CDP05_0001E&for=zip+code+tabulation+area%3A%2A&in=state%3A36
      zip  pop_total
0   10301        NaN
1   10302        NaN
2   10303        NaN
3   10304        NaN
4   10305        NaN
5   10306        NaN
6   10307        NaN
7   10308        NaN
8   10309        NaN
9   10310        NaN
10  10312        NaN
11  10314        NaN
12  11201        NaN
13  11203        NaN
14  11204        NaN
15  11205        NaN
16  11206        NaN
17  11207        NaN
18  11208        NaN
19  11209        NaN
20  11210        NaN
21  11211        NaN
22  11212        NaN
23  11213        NaN
24  11214        NaN
25  11215        NaN
26  11216        NaN
27  11217        NaN
28  11218        NaN
29  11219        NaN
30  11220        NaN
31  11221        NaN
32  11222        NaN
33  11223        NaN
34  11224        NaN
35  11225     

In [5]:
# Counts by ZIP from 311 noise
noise_counts = noise.groupby("zip").size().rename("noise_count").reset_index()

# Counts by ZIP from SLA active liquor licenses
lic_counts   = sla.groupby("zip").size().rename("license_count").reset_index()

# Merge ZIP population (right now pop_total is NaN fallback, which is OK),
# with noise_counts and license_counts
zip_table = (
    zcta_target[["zip","pop_total"]]
    .merge(noise_counts, on="zip", how="left")
    .merge(lic_counts,   on="zip", how="left")
)

# Clean up missing raw counts
zip_table["noise_count"]   = zip_table["noise_count"].fillna(0).astype(int)
zip_table["license_count"] = zip_table["license_count"].fillna(0).astype(int)

# Per-1,000 rates (will be NaN for now because pop_total is NaN,
# which is fine for this milestone)
zip_table["noise_per_1000"] = (zip_table["noise_count"] / zip_table["pop_total"] * 1000)
zip_table["licenses_per_1000"] = (zip_table["license_count"] / zip_table["pop_total"] * 1000)

# Borough label
zip_table["borough"] = zip_table["zip"].apply(borough_from_zip)

# Keep only Brooklyn + Staten Island
zip_table = zip_table[zip_table["borough"].isin(["Brooklyn","Staten Island"])]

# Sort to see the "loudest" ZIPs (by raw complaint count for now)
zip_table = zip_table.sort_values("noise_count", ascending=False)

print("Rows in merged table:", len(zip_table))
zip_table.head(15)


Rows in merged table: 51


,zip,pop_total,noise_count,license_count,noise_per_1000,licenses_per_1000,borough
36,11226,NaN,11288,0,NaN,NaN,Brooklyn
31,11221,NaN,10188,0,NaN,NaN,Brooklyn
16,11206,NaN,9878,0,NaN,NaN,Brooklyn
12,11201,NaN,8926,0,NaN,NaN,Brooklyn
21,11211,NaN,8835,0,NaN,NaN,Brooklyn
46,11237,NaN,8515,0,NaN,NaN,Brooklyn
22,11212,NaN,8384,0,NaN,NaN,Brooklyn
17,11207,NaN,7963,0,NaN,NaN,Brooklyn
35,11225,NaN,7383,0,NaN,NaN,Brooklyn
47,11238,NaN,6961,0,NaN,NaN,Brooklyn


In [7]:
CLEAN.mkdir(parents=True, exist_ok=True)
zip_table.to_csv(CLEAN/"zip_noise_licenses_bk_si.csv", index=False)
print("Saved merged table to data/clean/zip_noise_licenses_bk_si.csv")


Saved merged table to data/clean/zip_noise_licenses_bk_si.csv
